# Notebook to generate dataset of MPC I/O examples

In [ ]:
from mpc import *
from numpy.random import uniform as unf

In [ ]:
# X0 = np.array([9, 0.0, 0.0, 0.0, 0.0]) 
# X0 = np.array([3, 0.0, 0.0, 0.0, 0.0]) 
# X0 = np.array([0.1, 0.0, 0.0, 0.0, 0.0]) 

X0 = np.array([unf(3.0, 8.0), unf(-40, 40)*π/180, unf(-100, 100)*π/180, 0, 0])

In [ ]:
model = STM_model_dt_inputs(); 

ctrl = MPC_controller(